# In this notebook, we are going to use *BeatutifulSoup* to do some web scraping. Follow me, it's very cool!

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import numpy as np

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 

In [3]:
soup = BeautifulSoup(data, 'html5lib')

In [10]:
# print(soup.prettify())

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.string=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Voilà! See, that's what I am doing here.

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [31]:
df.shape

(103, 3)

# Let's try to get the coordinates by Geocoder.

In [12]:
!pip install geocoder
import geocoder # import geocoder

latitude = []
longitude = []

for postal_code in df['PostalCode']:

# initialize your variable to None
        lat_lng_coords = None

# loop until you get the coordinates
        while(lat_lng_coords is None):
             g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
             lat_lng_coords = g.latlng
        latitude = latitude.append(lat_lng_coords[0])
        longitude = longitude.append(lat_lng_coords[1])

KeyboardInterrupt: 

In [7]:
df2 = pd.read_csv('../..//Downloads/Geospatial_Coordinates.csv')
df2 = df2.rename(columns={'Postal Code': 'PostalCode'})

In [8]:
df = df.join(df2.set_index('PostalCode'), on='PostalCode', how='right').sort_index()

In [9]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [11]:
df.shape

(103, 5)